In [3]:
import nltk
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image 
from IPython.core.display import Image, display
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import pymongo
import datetime
import bson
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm
import random
from itertools import permutations
from nltk.tokenize import sent_tokenize, word_tokenize
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [4]:
import aerospike
config = {
    'hosts': [('35.228.136.58', 3000)]
}
client = aerospike.client(config).connect()
data_as_key = ("ids_to_validate", "data", "data")

tags_by_tag_old = client.get(data_as_key)[-1]['tags']

In [3]:
def updateAPP(bson_id, new_list):
    before = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["grafeed_confirmed"]    
    before_note = {"date": datetime.datetime.today(),
                   "tags": before,
                  }
    before_set = set(before)
    new_list_set = set(new_list)
    if len(before_set.union(new_list_set))!= 0 and len(before_set.intersection(new_list_set))/len(before_set.union(new_list_set)) != 1:
#         print(bson_id)
#         print("BEFORE:", before)
        db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(bson_id)},
                                    {"$set":{"grafeed_confirmed": new_list,
                                            "fix_grafeed_confirmed": before_note}})

        
        after = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["grafeed_confirmed"]    
#         print("AFTER:", after)
#         print(db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["fix_grafeed_confirmed"])

#     if len(before_set.union(new_list_set)) == 0:
#         print("LEN ZERO")
#         print("BEFORE:", new_list_set)
#         print("AFTER", before_set)

        
        
#         print({"FIX": before_note})
#         print()
#     print()
#     db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(bson_id)},
#                                     {"$set":{"grafeed_confirmed": new_list}})
#     print("AFTER:", db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["grafeed_confirmed"])
#     print("AFTER:", new_list)
#     print() 

In [5]:
def dictImplementation(tagsList):
    result = []
    for tag in tagsList:
        if tag.lower() in tags_by_tag_old and tags_by_tag_old[tag.lower()] != [0]:
            result += tags_by_tag_old[tag.lower()]
        if tag.lower() not in tags_by_tag_old:
            result.append(tag.lower())
    return list(set(result))

In [6]:
db = db_client["tweets_pipeline_v2"].find({"$or":[{"status": "graphicone_search"},{"status": "graphicone_feed"}]})
for i in tqdm(db, total = db.count()):    
    tagsFullSet = i["grafeed_confirmed"]
    updateAPP(str(i["_id"]), dictImplementation(tagsFullSet))
#     print()
#     print()

100%|██████████| 5668/5668 [02:39<00:00, 35.56it/s]


# VALIDATED

In [25]:
def dictImplementation(tagsList):
    result = []
    for tag in tagsList:
        if tag.lower() in tags_by_tag_old and tags_by_tag_old[tag.lower()] != [0]:
            result += tags_by_tag_old[tag.lower()]
        if tag.lower() not in tags_by_tag_old:
            result.append(tag.lower())
    return list(set(result))

def updateValidated(bson_id, new_list, mongo_key):
    before = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})[mongo_key]    
    before_note = {"date": datetime.datetime.today(),
                   "tags": before,
                  }
    before_set = set(before)
    new_list_set = set(new_list)

#     print(bson_id)
#     print("BEFORE:", before)

    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(bson_id)},
                                {"$set":{mongo_key: new_list,
                                        "fix_"+mongo_key: before_note}})

        
    after = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})[mongo_key]    
#     print("AFTER:", after)
#     print(db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["fix_"+mongo_key])
#     print()
#     print()

In [26]:
db = db_client["tweets_pipeline_v2"].find({"$or":[{"status": "validated"}]})
print(db.count())
# for i in tqdm(db, total = db.count()):    
key = "confirmed_after_validate"
for i in tqdm(db, total=db.count()):        
    tagsFullSet = i[key]
    updateValidated(str(i["_id"]), dictImplementation(tagsFullSet), key)

  0%|          | 0/1618 [00:00<?, ?it/s]

1618


100%|██████████| 1618/1618 [02:14<00:00, 12.05it/s]


In [27]:
db_client["tweets_pipeline_v2"].find({"$or":[{"status": "graphicone_search"},{"status": "graphicone_feed"}]}).count()

5668

# PIPELINED

In [3]:
def dictImplementation(tagsList):
    result = []
    for tag in tagsList:
        if tag.lower() in tags_by_tag_old and tags_by_tag_old[tag.lower()] != [0]:
            result += tags_by_tag_old[tag.lower()]
        if tag.lower() not in tags_by_tag_old:
            result.append(tag.lower())
    return list(set(result))

In [4]:
def updatePipelined(bson_id, bert_tags, entity_prepared):
#     print(bson_id)
    
    before = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(_id)})
    if "bert_tags" in before:
        before_bert = before["bert_tags"]
    else:
        before_bert = []
    before_entity_prepared = before["entity_prepared"] 
    
    #BERT
    before_bert_set = set(before_bert)
    bert_tags_set = set(bert_tags)
    
    if len(before_bert_set.union(bert_tags_set))!= 0 and len(before_bert_set.intersection(bert_tags_set))/len(before_bert_set.union(bert_tags_set)) != 1:
        
        db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(bson_id)},
                            {"$set":{"bert_tags": bert_tags}})


        
        print("BERT")
        print(before_bert)
        print(bert_tags)
        print()
    
    #ENTITY
    before_entity_prepared_set = set(before_entity_prepared)
    entity_prepared_set = set(entity_prepared)
    if len(before_entity_prepared_set.union(entity_prepared_set))!= 0 and len(before_entity_prepared_set.intersection(entity_prepared_set))/len(before_entity_prepared_set.union(entity_prepared_set)) != 1:
        
        db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(bson_id)},
                            {"$set":{"entity_prepared": entity_prepared}})
        
        print("ENTITY")
        print(before_entity_prepared_set)
        print(entity_prepared_set)
        print()
        

In [ ]:
db = db_client["tweets_pipeline_v2"].find({"$or":[{"status": "pipelined"},
#                                                   {"status": "reserved"}
                                                 ], 
                                           "aws_answer": {"$exists": True}})
for i in tqdm(db, total=db.count()):
#     print(i)
    _id = str(i["_id"])
    bert_tags = []
    entity_prepared = []
    if "bert_tags" in i:
        bert_tags = i["bert_tags"]
    if "entity_prepared" in i:
        entity_prepared = i["entity_prepared"]
        
    
    updatePipelined(_id, dictImplementation(bert_tags), dictImplementation(entity_prepared))    
    

    
    

 99%|█████████▊| 4365/4425 [02:02<00:01, 35.83it/s]

In [51]:
def dictImplementation(tagsList):
    result = []
    for tag in tagsList:
        if tag.lower() in tags_by_tag_old and tags_by_tag_old[tag.lower()] != [0]:
            result += tags_by_tag_old[tag.lower()]
        if tag.lower() not in tags_by_tag_old:
            result.append(tag.lower())                        
    return set(result)

In [69]:
counter = 0
checkSets = []
for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"},{"status":"graphicone_feed"}]}):
    analyst = set(dictImplementation(i["grafeed_confirmed"]))
    if "confirmed_after_validate" in i:
        validator = set(dictImplementation(i["confirmed_after_validate"]))
        if len(analyst.difference(validator)) > 0:
            print(i["_id"])
            counter += 1
            checkSets.append(str(i["_id"]))
        if len(validator.difference(analyst)) > 0:
#             print(i["_id"])
            counter += 1
            checkSets.append(str(i["_id"]))

len(set(checkSets))
    

5dda4459ecf2eeb7be924820
5dfbfeb1bf2e0b3cf6aecb6f
5e06bf901d9672b9e003d383
5e171b8b721658904464fd1a
5e17522a3a3af4aa7fe0c4ee
5e18315b9b4b433e300a554a
5e1877ef08402d836a4aece3
5e1900d74c782d7df714658b
5e1a80e29cabbc6997e87f3a
5e1b8574240606d70853fc5f
5e1f2451040ade4c827f233d
5e1f8fe8ab9a6a872fc699fc
5e1f9f710a29264b5020c85d
5e1fbbfa4ad9f4ab57e1cd40
5e1fbc4367352fa383e25b85
5e1ff0351d9798f55a628ab0
5e20575fe4714ac78e6ab101
5e20fc2b0c127b5c816a948c
5e21471f0b9409f4aa08cc86
5e22b4db0824aa0da86ccaa2
5e234c02223ba329831a3027
5e24612bb21e37b7398d8228
5e25561b6a456bfd3470213d
5e257cd309bb88be0dfec14a
5e25837f81b38666a5a1fdd4
5e2900c77aac96bdd8f49d6b
5e2ae3bd6185054b073a06dc
5e2b4e96104501b05c52c74a
5e2b73c66aaf40e2fdc111e8
5e2b7ac26e976c0dd1f8ab0c
5e2ba6204e98a41e99a653c9
5e2beead82eca6a863a18c4b
5e2bf6d05b1120a89498bef0
5e2ca381ee63d0fbe956e3ce
5e2e592114ea8c5601407eaf
5e2eb5a9a5638d02783df750
5e2edee478b7016d820ce32f
5e2f3243d82132d1be604816
5e2f9c7fd29da1713efd2004
5e2fb4dfd82132d1be604b2d


94

In [9]:
checkSets

['5dd12abfedee5484619cc1b4',
 '5dd14c1de008481920c2948a',
 '5dd197b6ff336ebb4bcdd13c',
 '5dd198fdff336ebb4bcdd1ac',
 '5dd34a1bafd3a2104cea9c20',
 '5dd34da3afd3a2104cea9de5',
 '5dd3b6c43e616f8117162377',
 '5dd3c0e13e616f811716269a',
 '5dd792b61167352c5ccea554',
 '5dd797451167352c5ccea717',
 '5dd9c774e321ac72a79fb521',
 '5dd9d98c99093d6f05111e22',
 '5dda41c4ecf2eeb7be92471c',
 '5dda499b1d1484b8f03e4a1c',
 '5ddaa27097a2dbd5c9889ede',
 '5ddab68786facb54ae6c5fbb',
 '5ddad71486facb54ae6c6d9e',
 '5ddb24a2120fcc7581548bf4',
 '5ddb6fc6120fcc7581549f18',
 '5ddb92c2ed18622ca1f2f4ec',
 '5ddbd677ecf474e943a140e6',
 '5ddbdcdaecf474e943a14376',
 '5ddc18d62c13a93c09a9bf0c',
 '5ddc1ec22c13a93c09a9c1f0',
 '5ddc2c941fdcd26a2b7b55e3',
 '5ddc9b53bd6f7b1fda25f61f',
 '5ddcad26bd6f7b1fda25fc68',
 '5de4f7c182607ee2ef4ada0b',
 '5de50e57879e307cf7a64934',
 '5de5120beaa6d9fa7f8c5614',
 '5de5161d6965f1632be4292e',
 '5de51944f6b77606f742a83e',
 '5de519c282607ee2ef4ae4fa',
 '5de51d3ce08ae675e9d7f528',
 '5de51fa441d1

In [10]:
data_validator = []
data_analyst = []
for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"},{"status":"graphicone_feed"}]}):
    data_analyst += dictImplementation(i["grafeed_confirmed"])    
    if "confirmed_after_validate" in i:
        data_validator += set(dictImplementation(i["confirmed_after_validate"]))
    
    
    

In [11]:
val = pd.DataFrame.from_dict(Counter(data_validator).most_common())
an = pd.DataFrame.from_dict(Counter(data_analyst).most_common())

In [12]:
val.merge(an, on=[0], how="outer").to_excel("2.xlsx")

In [59]:
def update_tweet_app(bson_id, new_list):
    print(bson_id)
    approved_before = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["grafeed_confirmed"]
    
    before_note = {"date": datetime.datetime.today(),
                   "list": approved_before,}
#     print(new_list)
    db_client["tweets_pipeline_v2"].update_one({"_id": bson.ObjectId(bson_id)},
                                        {"$set":{"grafeed_confirmed": new_list,
                                                 "fix": before_note,
                                                 "confirmed_after_validate": new_list}})
    
    approved_after = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["grafeed_confirmed"]
    print(f"{approved_before} -> {approved_after}")

In [63]:
searchAndFeed = pd.read_excel("errorsSearchFeed.xlsx")
searchAndFeed = list(searchAndFeed["ID"].values)
print(len(searchAndFeed))
print(searchAndFeed)

924
['5df03a1bb6444b472c8477bc', '5e2b01b45706a71c1768cfce', '5dfb98f8e37cc217ac91edc2', '5dfb83cb26947026f22994f0', '5e1c8bd8331f0686415ee081', '5ddab84986facb54ae6c607f', '5dfb83cf26947026f2299515', '5dfb7ee7a58cebdca7291484', '5dfb7beb51e9d21cd70eca5a', '5ddc0d67ecf474e943a15027', '5dfbc99d20afaad67ff26987', '5de63bbcd7ef823cc420bd8a', '5dda81c097a2dbd5c9889208', '5ddb5856120fcc75815496c2', '5ddb224c120fcc7581548b31', '5e0312be53d8049a4a96056d', '5ddaca26b3580e6d5d3e13f4', '5e2c04257f204816bfb57c12', '5e2e254ab084db7a3921abb5', '5de5020a50452a49d140ef6e', '5ddb24c1120fcc7581548c28', '5ddbfdececf474e943a14c91', '5dfc3b2ee75f75b47f6029fa', '5dfbfc3c7931d329d8feece8', '5de5286f3bed7af1ba1abcd3', '5dfb67e23ddc874216676d8e', '5dfc6b4aaaa9751df222679e', '5df23948ab8c17d1620c17d9', '5df038286cdcbbbb8c789961', '5dfb7a6d00057b115ae1a5e1', '5deff201c1bd5a6cafc9275a', '5e02af9065c864d5a7805908', '5dda9a98878dddefe2d56949', '5e0373a9b0646dbaae2ab7f5', '5df23936ab8c17d1620c1790', '5e020d7a9a4a60

In [67]:
counter = 0
for _id in searchAndFeed:
    if "fix" not in db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(_id)}):
        counter += 1
        update_tweet_app(_id, db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(_id)})["grafeed_confirmed"])
        
counter        

0

In [6]:
import re

In [15]:
db = db_client["tweets_pipeline_v2"].find({"$or":[{"status": "graphicone_search"},{"status": "graphicone_feed", "status": "validated"}]})
for i in tqdm(db, total = db.count()):    
    if "grafeed_confirmed" in i:
        tagsFullSet = i["grafeed_confirmed"]    
        for tag in tagsFullSet:
            if re.search("johnson", tag):
                print(tagsFullSet)
    elif "confirmed_after_validate" in i:
        tagsFullSet = i["confirmed_after_validate"]    
        for tag in tagsFullSet:
            if re.search("johnson", tag):
                print(tagsFullSet)    
    else:
        print(i)

  1%|          | 102/9158 [00:00<11:59, 12.58it/s]

['wealth', 'chevron', 'altria', 'walmart', 'exxon', 'at&t', 'microsoft', 'general motors', 'us', 'amazon', 'general electric', 'apple', 'procter&gamble', 'johnson & johnson', 'ibm', 'coca-cola']
['google', 'chevron', 'alibaba', 'china', 'us', 'general electric', 'apple', 'ipo', 'wells fargo', 'johnson & johnson']
['exxon', 'shell', 'facebook', 'gazprom', 'china mobile', 'industrials', 'alphabet', 'health', 'energy', 'general electric', 'apple', 'petrochina', 'berkshire hathaway', 'bp', 'bank of america', 'financials', 'johnson & johnson', 'citigroup', 'telecom', 'microsoft', 'amazon', 'information technology', 'hsbc']
['ftse', 'funds', 'vanguard', 'etf', 'bonds', 'developed markets', 'facebook', 'vti', 'personal finance', 'at&t', 'general electric', 'apple', 'us', 'johnson & johnson', 'google', 'intel', 'microsoft', 'verizon', 'emerging markets', 'index']
['bayer', 'servier', 'consumer staples', 'manufacturing', 'health', 'czech republic', 'zentiva', 'medicine', 'johnson & johnson', 'd

100%|██████████| 9158/9158 [00:01<00:00, 6507.01it/s]

['pfizer', 'google', 'cisco', 'cash', 'microsoft', 'exxon', 'us', 'general electric', 'apple', 'johnson & johnson', 'ibm', 'merck']
['petrochina', 'facebook', 'mastercard', 'dutch east india company', 'alibaba', 'berkshire hathaway', 'jpmorgan', 'walmart', 'microsoft', 'alphabet', 'south sea company', 'bank of america', 'amazon', 'saudi aramco', 'apple', 'procter&gamble', 'johnson & johnson', 'visa']
['pfizer', 'abbvie', 'roche', 'pharmaceutical industry', 'sanofi', 'amgen', 'bristol-myers squibb', 'novartis', 'johnson & johnson', 'merck']
['health', 'energy', 'jpmorgan', 's&p', 'exxon', 'large caps', 'lockheed martin', 'industrials', 'vanguard', 'index', 'us', 'technology', 'consumer discretionary', 'amazon', 'apple', 'las vegas sands', 'host hotels & resorts', 'microsoft', 'blackrock', 'telecom', 'consumer staples', 'financials', 'materials', 'utilities', 'johnson & johnson']
['facebook', 'jpmorgan', 'microsoft', 'exxon', 'amazon', 'apple', 'johnson & johnson', 'index', 's&p']
['alph

In [5]:
for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"},{"status":"graphicone_feed"}]}):
    print(i)

{'_id': ObjectId('5dc4c2565dcf6274b6a44809'), 'tweet_text': "The World's 20 Most Profitable Companies 💵\nhttps://t.co/fY7YXoAixd via @VisualCap #money #business #economy #wealth https://t.co/GPqtI7LVLY", 'hashtags': ['money', 'business', 'economy', 'wealth'], 'tweet_timestamp': datetime.datetime(2019, 10, 27, 16, 2), 'username': 'tamaramccleary', 'user_id': 2307675307, 'user_counters': {'followers_count': 306374, 'friends_count': 215979, 'listed_count': 8590}, 'tweet_counters': {'retweet_count': 11, 'favorite_count': 11}, 'img_urls': 'http://pbs.twimg.com/media/EHz92SGVUAAXDDB.jpg', 'tweet_id': 1188486009236836358, 'downloaded': True, 'download_error': False, 'img_type': 'graphic', 'graph_type': 'other-chart', 'confirmed_after_validate': ['saudi aramco', 'apple', 'icbc', 'samsung', 'china construction bank', 'jpmorgan', 'alphabet', 'bank of america', 'shelll', 'shell', 'wells fargo', 'gazprom', 'facebook', 'intel', 'exxonmobil', 'at&t', 'citi', 'toyota'], 'status': 'graphicone_feed', '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'_id': ObjectId('5ed9003cd57084171a5de6b4'), 'tweet_text': 'Valuation of growth relative to value now above 1999 levels https://t.co/bHK4NTNyiP', 'hashtags': [], 'username': 'steveclapham', 'tweet_timestamp': datetime.datetime(2020, 6, 2, 8, 5, 54), 'tweet_id': 1267729139412451329, 'img_urls': 'https://pbs.twimg.com/media/EZfgblSXQAEWuff.jpg', 'TIME_FIELD_FOR_SORTING': datetime.datetime(2020, 6, 2, 8, 5, 54), 'source': 'twitter', 'img_type': 'graphic', 'tech_type': 'norm', 'status': 'graphicone_search', 'extracted_text': 'StarCapital Research Relative overvaluation Growth vs. Value exceeds New Economy levels! MSCI World Growth premium since 1974 in uncharted waters 200% -Growth Valuation Premium (MSCI World Growth vs. MSCIWorld Value) April 2020: 182% 170% Growth Premium in 2000: 177% 140% 110% Average 82% 80% 50% 20% Dec-74 Dec-79 Dec-84 Dec-89 Dec-94 Dec-99 Dec-04 Dec-09 Dec-14 Dec-19 This chart shows the overvaluation of the MSCI World Growth Index compared to the MSCI World Value 